# SEIR model with time-varying $\beta$

**References**

[Epidemic Modeling 101: Or why your CoVID-19 exponential fits are wrong](https://medium.com/data-for-science/epidemic-modeling-101-or-why-your-covid19-exponential-fits-are-wrong-97aa50c55f8)

[Epidemic Modeling 102: All CoVID-19 models are wrong, but some are useful](https://medium.com/data-for-science/epidemic-modeling-102-all-covid-19-models-are-wrong-but-some-are-useful-c81202cc6ee9)

[Compartmental models in epidemiology](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology)

## Background

We want to explore the effect of interventions in an epidemic. We use a standard epidemiological SEIR model (see references above), and model the effect of interventions as reducing the infection coefficient $\beta$.

The equations are
$$\begin{align}
\frac{\partial S_t}{\partial t} &= -\beta S_t \frac{I_t}{N} \\
\frac{\partial E_t}{\partial t} &= \beta S_t \frac{I_t}{N} - \alpha E_t\\
\frac{\partial I_t}{\partial t} &= \alpha E_t - \mu I_t \\
\frac{\partial R_t}{\partial t} &= \mu I_t
\end{align}$$

where
$$\begin{align}
\alpha^{-1} &= \text{average incubation period} \\
\beta &= \text{rate of infection} \\
\mu &= \text{rate of recovery}
\end{align}$$

Now 
$$R = \frac{\beta}{\mu}$$

is the reproduction number. (We'll use the notation $R$ without a subscript for the reproduction number to avoid confusion with $R_t$, the number of recovered individuals.) A basic result is that the number of people unaffected by the disease $S_\infty$ can be determined from the relation

$$S_\infty =e^{-R (1-S_\infty)}$$

A guess for the value of $R$ for COVID-19 is 2.5. We will model the effect of measures like physical distancing as reducing $\beta$ and hence $R$ over time. We will therefore regard them as time-dependent quantities, $R_t$ and $\beta_t$.

Although in some ways it is only the ratio $\beta / \mu$ that matters, the magnitudes of $\beta$ and $\mu$ affect how quickly the epidemic unfolds.

To incorporate these ideas into the numerical integration, we will specify two times (step numbers), $t_1$ and $t_2$, in addition to the total number of steps $t_f - 1$. Then $\beta_t$ has the value $\beta_i$ for $t\le t_1$, $\beta_f$ for $t \ge t_2$ and ramps linearly in between.

In [ ]:
%matplotlib inline

from datetime import date
from io import StringIO

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import requests
from sklearn.linear_model import LinearRegression
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, least_squares

plt.style.use('seaborn')

In [ ]:
def r0_to_infected(r0):
    """Compute limit value of population infected"""
    def func(Sinf, r0):
        return Sinf - np.exp(-r0 * (1 - Sinf))
    
    return 1 - fsolve(func, 0.5, args=r0)[0]

In [ ]:
def backcast(y, n):
    """Use linear regression to extrapolate values to before start of time series"""
    X = np.arange(len(y)).reshape(-1, 1)
    model = LinearRegression()
    model.fit(X, y)
    X_predict = np.arange(-n, 0).reshape(-1, 1)
    return model.predict(X_predict)


In [ ]:
def calc_new_cases(cases_cum):
    """Calculate new cases based on cumulative cases. New cases are the total cases in the previous week."""
    # Do some funny business to impute weekly sums at beginning of time series
    cases_ext = backcast(np.log(cases_cum[:7] + 1e-7), 7)
    cases_ext = np.exp(cases_ext)

    cases_new = np.zeros_like(cases_cum)
    cases_new[:7] = cases_cum[:7] - cases_ext
    cases_new[7:] = cases_cum[7:] - cases_cum[:-7]
    
    return cases_new

## Example of modeled data
This is a model with fixed parameters just to see what the outputs look like and how they look in the log-log visualization.

In [ ]:
# Equations to be integrated
t0 = 0
t1 = 60  # start of interventions [20]
t2 = t1 + 20  # interventions have stabilized [60]
t3 = 180  # days to run model for [365]
r1 = 2.0  # initial R [2.5]
r2 = 0.9 # final R [0.5]
mu = 1 / 7  # 1 / average recovery time [1 / 14]
beta1 = r1 * mu
beta2 = r2 * mu
alpha = 1 / 7  # 1 / average incubation period [1 / 7]
N = 5.1e6  # population [1e6]
E0 = 100  # number initially exposed [N * 0.01]
I0 = 100  # number initially infected [1]
S0 = N - E0 - I0  # number initially susceptible
R0 = 0  # number initially recovered

# t1 = 19.503179
# t2 = 31.759004
# mu = 14.349860
# beta1 = 14.904735
# beta2 = 14.299944
# alpha = 6.418785
# E0 = 18.563979
# I0 = 0.01383

# N = 60.36e6
# t0 = 0
# t3 = 180  # 60
# R0 = 0
# S0 = N - I0 - E0 - R0

# Ref: http://www.public.asu.edu/~hnesse/classes/seir.html
# t0 = 0
# t1 = 0
# t2 = 0
# t3 = 180
# beta1 = 0.9
# beta2 = beta1
# mu = 0.2  # Gamma
# alpha = 0.5  # Sigma
# S0 = 10
# E0 = 1
# I0 = 0
# R0 = 0
# N = S0 + E0 + I0 + R0

def beta(t):
    if t <= t1:
        return beta1
    if t <= t2:
        return beta2 - (t2 - t) * (beta2 - beta1) / (t2 - t1)
    return beta2


def fun(t, y):
    dy_dt = np.zeros_like(y)
    S, E, I, R = y
    dS_dt = -beta(t) * S * I / N
    dE_dt = beta(t) * S * I / N - alpha * E
    dI_dt = alpha * E - mu * I
    dR_dt = mu * I
    return dS_dt, dE_dt, dI_dt, dR_dt
    

In [ ]:
# Solve the equations (numerically)
bunch = solve_ivp(fun, t_span=(t0, t3), y0=(S0, E0, I0, R0), t_eval=np.arange(t3 + 1))
S, E, I, R = bunch.y
t = bunch.t
Imax = np.max(I) / N
Iarg = np.argmax(I)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title(f'Maximum infections: {Imax:.1%} at {Iarg} days')
ax.plot(t, S / N, label='Susceptible')
ax.plot(t, E / N, label='Exposed')
ax.plot(t, I / N, label='Infected')
ax.plot(t, R / N, label='Recovered')
ax.legend()
plt.show()

In [ ]:
cases_cum = E + I + R
plt.plot(t, cases_cum, label='Cumulative cases')
plt.legend()
plt.show()

In [ ]:
cases_new = calc_new_cases(cases_cum)

fig, ax = plt.subplots()
ax.scatter(cases_cum, cases_new)
# ax.set_ylim(bottom=1)
ax.set_xlabel('Cumulative cases')
ax.set_ylabel('New weekly cases')
ax.set_yscale('log')
ax.set_xscale('log')
for i in range(0, t3, 10):
    ax.annotate(f'{i}', (cases_cum[i], cases_new[i]), (cases_cum[i] * .8, cases_new[i] * 1.2), c='darkgreen')
plt.tight_layout()
plt.show()